#### 라이브러리 및 환경변수 세팅

In [2]:
import numpy as np
import pandas as pd
import os
from google.analytics.data_v1beta import BetaAnalyticsDataClient 
from google.analytics.data_v1beta.types import (
    RunReportRequest, 
    Dimension, 
    Metric,
    OrderBy, 
    DateRange, 
)

In [ ]:
# os 모듈을 사용하여 환경 변수를 설정할 수 있습니다.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'GCP에서_생성한_json_파일.json'
# Google Analytics 4 속성 ID 설정
property_id = 'GA4_속성_ID'
# Google Analytics Data API 클라이언트 초기화. 이 클라이언트를 통해 API에 요청을 보내고 데이터를 받을 수 있습니다.
client = BetaAnalyticsDataClient()

### 측정기준, 측정항목 세팅
- 측정기준: GA4 측정기준은 이벤트이름, 페이지위치, 플랫폼 등 GA4에서 제공하는 Columns같은 개념이다.<br>
실제로 빅쿼리에서는 측정기준이 Columns로 되어있다.

- 측정항목: GA4 측정항목은 액티브유저수, 이벤트수 등 GA4에서 제공하는 집계 함수같은 개념이다.<br>
raw 데이터를 groupby하여 .sum(), .mean()과 같은 함수로 집계를 하는 것처럼 GA4도 측정항목을 통해 집계를 할 수 있다.

사용할 수 있는 측정기준, 측정항목 값은 [GA4 API 공식 문서](https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema?hl=ko)에서 찾을 수 있다.<br>
해당 문서에 나와있는 API 이름을 모두 사용할 수 있다.<br>

예시로 일별 DAU를 출력해보자. 이 경우엔 측정기준을 'date'로, 측정항목을 'active1DayUsers'로 설정하면 된다.

In [6]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="date")],
        metrics=[Metric(name="active1DayUsers")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'date'})], 
        date_ranges=[DateRange(start_date="2024-01-01", end_date="today")],
    )

In [7]:
response = client.run_report(request)

In [8]:
data = [] 

for row in response.rows:
    row_data = {dim.name: row.dimension_values[i].value for i, dim in enumerate(response.dimension_headers)}
    row_data.update({metric.name: float(row.metric_values[i].value) for i, metric in enumerate(response.metric_headers)})
    data.append(row_data)
df = pd.DataFrame(data)
df.head(1)

,date,active1DayUsers
0,20240101,63669.0
